# 学习笔记TF029:实现进阶卷积网络

经典数据集CIFAR-10,60000张32x32彩色图像，训练集50000张，测试集10000张。标注10类，每类图片6000张。airplance、automobile、bird、cat、deer、dog、frog、horse、ship、truck。没有任何重叠。CIFAR-100,100类标注。深度学习之父 Geoffrey Hinton和学生Alex Krizhevsky、Vinod Nair收集。图片源于80 million tiny images数据集。State-of-the-art 3.5%错误率，GPU训练十几小时。详细Benchmark和排名在 http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html 。LeCun，现有卷积神经网络已经解决CIFAR-10数据集问题。

根据Alex cuda-convnet模型修改，3000个batch，每个batch 128个样本，达到73%正确率。GTX1080单显卡几十秒模型训练时间。CPU慢很多。如用100k batch 结合学习速度decay(每隔一段时间下降学习速率一个比率)，正确率可到86%。模型训练参数100万个，预测四则运算总量2000万次。对weights进行L2正则化。图片翻转、随机剪切等数据增强，制造更多样本。每个卷积-最大池化层后用LRN层，增强模型泛化能力。

下载TensorFlow Models库，使用其中提供CIFAR-10数据类。git clone https://github.com/tensorflow/models.git。models/tutorials/image/cifar10。

载入常用库，NumPy、time，TensorFlow Models自动下载、读取CIFAR-10数据类。

定义batch_size，训练轮数max_steps，下载CIFAR-10数据默认路径。

定义初始化weight函数，tf.truncated_normal截断正态分布初始化权重。Weight加L2 loss ,做L2 正则化。减少特征或惩罚不重要特征权重，缓解特征过多导致过拟合。正则化帮助找到该惩罚的特征权重。为使用某个特征，需付出loss代价。L1正则制造稀疏特征，大部分无用特征权重被置0。L2正则让特征权重不过大，特征权重较平均。wl控制L2 loss大小，tf.nn.l2_loss函数计算weight L2 loss，tf.multiply L2 loss 乘以wl,得最后 weight loss。tf.add_to_collection weight loss统一存在collection losses，计算神经网络总体loss使用。

用cifar10类下载数据集，解压、展开到默认位置。

用cifar10_input类 distorted_inputs函数产生训练数据，包括特征、label，返回封装tensor，每次执行生成一个batch_size数量样本。Data Augmentation(数据增强),cifar10_input.distorted_inputs函数，随机水平翻转(tf.image.random_flip_left_right)、随机剪切一块24x24图片(tf.random_crop)、设置随机亮度对比度(tf.image.random_brightness、tf.image.random_contrast)，数据标准化(tf.image.per_image_whitening，数据减均值，除方差，保证数据零均值，方差1)。获得更多样本，带噪声，一张图片样本变多张图片，扩大样本量，提高准确率。数据增强操作耗费大量CPU时间，distored_inputs用16个独立线程加速任务，函数内部产生线程池，通过TensorFlow queue调度。

用cifar10_input.inputs函数生成测试数据，裁剪图片正中间24x24大小区块，数据标准化。

创建输入数据placeholderx，特征、label。设定placeholder数据尺寸，batch_size定义网络结构要用，数据尺寸第一个值样本条数需要预先设定，不能设None。数据尺寸的图片尺寸为24x24,裁剪后大小，颜色通道数3,彩色RGB三通道。

第一个卷积层，variable_with_weight_loss 函数创建卷积核参数初始化。卷积核大小5x5,3个颜色通道，64个卷积核，设置weight初始化函数标准差0.05。wl(weight loss)设0。tf.nn.conv2d函数对输入数据image_holder卷积操作，步长stride设1,padding模式SAME，bias初始化0，卷积结果加bias，用ReLU激活函数非线化。用尺寸3x3,步长2x2最大池化层处理数据，尺寸、步长不一致，增加数据丰富性。tf.nn.lrn函数，LRN，处理结果。

LRN起于Alex用CNN参加ImageNet比赛论文。LRN模仿生物神经系统侧抑制机制，对局部神经元活动创建竞争环境，响应较大值变得相对更大，抑制其他反馈较小神经元，增强模型泛化能力。用LRN后CNN Top1错误率降低1.4%。LRN对无上限边界激活函数ReLU有用，从附近多个卷积核响应(Response)挑选较大反馈，不适合固定边界能抑制过大值激活函数Sigmoid。

第二个卷积层，卷积核尺寸第三维度输入通道数64,bias值全初始化0.1。先进行LRN层处理，再用最大池化层。

全连接层，把前面两个卷积层输出结果全部flatten，tf.reshape函数把每个样本变成一维向量。get_shape函数获取数据扁平化长度。variable_with_weight_loss函数初始化全连接层weight，隐含节点384,正态分布标准差0.04，bias初始化0.1。设非零weight loss值0.04，所有参数被L2正则约束，避免过拟合。ReLU激活函数非线性化。

第二个全连接层，隐含节点192。

最后一层，先创建weight，正态分布标准差设上一隐含层节点数倒数，不计入L2正则。Softmax操作放在计算loss部分，不需要对inference输出softmax处理，就可以获得最终分类，直接比较inference输出各类数值大小。

整个卷积神经网络从输入到输出流程。设计CNN，安排卷积层、池化层、全连接层分布和顺序，超参数设置、Trick使用。卷积神经网络结构：
conv1:卷积层和ReLU激活函数
pool1:最大池化
norm1:LRN
conv2:卷积层和ReLU激活函数
norm2:LRN
pool2:最大池化
local3:全连接层和ReLU激活函数
local4:全连接层和ReLU激活函数
logits:模型Inference输出结果

计算CNN loss。softmax计算和cross entropy loss 计算合在一起，tf.nn.sparse_softmax_cross_entropy_with_logits。tf.reduce_mean计算cross entropy均值，tf.add_to_collection 添加cross entropy loss 到整体losses collection。tf.add_n整体losses collection 全部loss求和，得最终loss，包括cross entropy loss，和后两个连接层weight L2 loss。Logits节点、label_placeholder传入loss小孩子数，获得最终loss。

优化器选择Adam Optimizer，学习速率1e-3。

tf.nn.in_top_k函数求输出结果top k准确率，默认top 1,输出分类最高类准确率。

tf.InteractiveSession创建默认session ,初始化全部模型参数。

启动图片数据增强线程队列，16个线程加速。

训练。每个step训练过程，session run方法执行images_train、 labels_train计算，获得batch训练数据，传入train_op和loss计算。记录每个step时间，每隔10个step计算展示当前loss、每秒钟训练样本数量、训练batch数据时间，监控整个训练过程。GTX 1080,每秒训练1800个样本，batch_size 128,每个batch 0.066s。损失loss，开始4.6，3000步训练下降到1.0。

评测模型测试集准确率。测试集10000个样本，使用固定batch_size，逐个batch输入测试数据。计算全部样本评测完batch数量。每个step用session run方法获取images_test、labels_test的batch，执行top_k_op计算模型 batch top 1预测正确样本数。汇总所有预测正确结果，求全部测试样本预测正确数量。

打印准确率评测结果计算。

73%准确率。持续增加max_steps，期望准确率逐渐增加。max_steps较大，用学习速率衰减(decay)的SGD训练，接近86%。L2正则，LRN层提升模型准确率，提升框泛化性。

数据增强(Data Augmentation)，给单幅图增加多个副本，提高图片利用率，防止图片结构学习过拟合。利用图片本身性质，图片冗余信息量较大，制造不同噪声，依可识别。神经网络克服噪声准确识别，泛化性更好。深度学习只要提供足够多样本，准确率可以持续提升。 规模越大越复杂神经网络模型，可以达到准确率水平越高，需要更多数据训练。Alex cuda-convnet测试结果，CIFAR-10,不数据增强，错误最低下降到17%，数据增强，错误率下降到11%。

In [4]:
import cifar10, cifar10_input

usage: ipykernel_launcher.py [-h] [--batch_size BATCH_SIZE]
                             [--data_dir DATA_DIR] [--use_fp16 USE_FP16]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\qiang\AppData\Roaming\jupyter\runtime\kernel-d06a8a6b-410e-451c-9eef-e7661d8a8b6c.json


SystemExit: 2

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
cifar10.maybe_download_and_extract()

NameError: name 'cifar10' is not defined